In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

# Define AutoGen Agent 

In [2]:
from autogen import ConversableAgent 
# define agent 
agent = ConversableAgent(
    name="chatbot", #name of the agent 
    llm_config=llm_config, # pass the llm model
    human_input_mode="NEVER" # agent will never seek for human input 
    # if we pass "ALWAYS" then the agent will always ask for human input before it tries to generate reply on his own 
)

In [3]:
# generate reply based on content given 
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)

print(reply)

Sure! Here's a joke for you:

Why couldn't the bicycle stand up by itself?
Because it was two tired!


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke", "role": "user"}]
)

print(reply)

Of course! Please provide me with the joke you'd like me to repeat for you.


# Conversation 
- Setting up a conversation between two agents 
- The conversation interaction memory is retained 

In [5]:
# create 2 agent as stan-up comedian 

cathy = ConversableAgent(
    name="cathy",
    system_message="Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

joe = ConversableAgent(
    name="joe",
    system_message="Your name is Joe and you are a stand-up comedian."
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode='NEVER'
)

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let's keep the laughs coming. So I recently started a new diet where I eat a salad every day. Not because I want to be healthy, but because it makes me feel like I'm eating a cheeseburger while I cry on the inside.

--------------------------------------------------------------------------------
joe (to cathy):

That's right, Cathy. It's all about that emotional eating balance. Speaking of diets, I tried to give up carbs once. It was a nightmare. I realized that without carbs, I'm basically just a shell of a person.

--------------------------------------------------------------------------------
cathy (to joe):

It's so true, Joe! Carbs are like the glue that holds our lives together. Without them, we're just wandering aimlessly, probably in the direction of the nearest bakery. And let's face it, who wants t

# Print result 
- Chat history
- Cost 
- Summary of the conversation 

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Hey Joe! Alright, let's keep the laughs coming. So I recently "
             'started a new diet where I eat a salad every day. Not because I '
             "want to be healthy, but because it makes me feel like I'm eating "
             'a cheeseburger while I cry on the inside.',
  'name': 'cathy',
  'role': 'user'},
 {'content': "That's right, Cathy. It's all about that emotional eating "
             'balance. Speaking of diets, I tried to give up carbs once. It '
             "was a nightmare. I realized that without carbs, I'm basically "
             'just a shell of a person.',
  'name': 'joe',
  'role': 'assistant'},
 {'content': "It's so true, Joe! Carbs are like the glue that holds our lives "
             "together. Without them, we're just wandering aimlessly, probably "
             "in the direction of the nearest bakery. And let's face it, who "
         

In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 158,
                                                             'cost': 0.000388,
                                                             'prompt_tokens': 302,
                                                             'total_tokens': 460},
                                      'total_cost': 0.000388},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 158,
                                                             'cost': 0.000388,
                                                             'prompt_tokens': 302,
                                                             'total_tokens': 460},
                                      'total_cost': 0.000388}}


In [9]:
pprint.pprint(chat_result.summary)

("It's so true, Joe! Carbs are like the glue that holds our lives together. "
 "Without them, we're just wandering aimlessly, probably in the direction of "
 "the nearest bakery. And let's face it, who wants to live in a world without "
 'breadsticks?')


# Get a better summary of the conversation 

In [10]:
chat_result = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation"
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let's keep the laughs coming. So I recently started a new diet where I eat a salad every day. Not because I want to be healthy, but because it makes me feel like I'm eating a cheeseburger while I cry on the inside.

--------------------------------------------------------------------------------
joe (to cathy):

That's right, Cathy. It's all about that emotional eating balance. Speaking of diets, I tried to give up carbs once. It was a nightmare. I realized that without carbs, I'm basically just a shell of a person.

--------------------------------------------------------------------------------
cathy (to joe):

It's so true, Joe! Carbs are like the glue that holds our lives together. Without them, we're just wandering aimlessly, probably in the direction of the nearest bakery. And let's face it, who wants t

In [11]:
pprint.pprint(chat_result.summary)

('Joe and Cathy share jokes about their struggles with diets and the '
 'importance of carbs in their lives. They humorously highlight the challenges '
 'of giving up carbs and how they serve as emotional support in their diets.')


# Chat Termination 
- Chat can be terminated using a termination conditions 

In [17]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [18]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! What do you call a fake noodle? An impasta!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, classic one, Cathy! What did one wall say to the other wall? "I'll meet you at the corner!"

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I love it! Walls have great set-up and delivery, don't they?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, they sure do! They always know how to keep a joke structured!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, they're very rigid that way!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, exactly! They re

In [19]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was how walls know how to hold up their end of puns.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's right! Thanks for the reminder. Well, it was great chatting with you, Joe! Have a fantastic day!

--------------------------------------------------------------------------------
joe (to cathy):

You too, Cathy! Keep spreading the laughter! Bye!

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go

--------------------------------------------------------------------------------
